In [4]:
# Import Commands
!pip install -q transformers accelerate bitsandbytes
!pip install -U bitsandbytes
!pip install perplexityai
!pip install flask pyngrok requests
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch 
import os
import json
from flask import Flask, request, jsonify
import requests
from datetime import datetime
import random
import re
from kaggle_secrets import UserSecretsClient
from pyngrok import ngrok
import threading
import re
import json
from perplexity import Perplexity
import time



In [5]:
# Scam Detector using Perplexity 

SCAM_KEYWORDS = [
    "urgent", "otp", "verify", "blocked", "suspend",
    "account compromised", "reward", "lottery",
    "cashback", "refund", "limited time",
    "click here", "act now", "bank", "upi",
    "password", "pin", "transfer", "crypto",
    "investment", "support team", "link",
        "refund",
        "prize",
        "lottery"
]

SENSITIVE_PATTERNS = [
    r"\b\d{4,6}\b",          # OTP-like
    r"\b\d{10,18}\b",        # Account-like numbers
    r"http[s]?://",          # URLs
    r"\b[\w\.-]+@[\w\.-]+\b" # UPI / email
]


def rule_score(message: str) -> float:
    score = 0.0
    lower_msg = message.lower()

    # Keyword scoring
    for keyword in SCAM_KEYWORDS:
        if keyword in lower_msg:
            score += 0.1

    # Pattern scoring
    for pattern in SENSITIVE_PATTERNS:
        if re.search(pattern, message):
            score += 0.2

    # Urgency signals
    if "!" in message:
        score += 0.05

    return min(score, 1.0)


def safe_json_parse(text: str) -> dict:
    try:
        cleaned = text.replace("```json", "").replace("```", "").strip()
        parsed = json.loads(cleaned)

        # Force correct types
        return {
            "is_scam": bool(parsed.get("is_scam", False)),
            "confidence": int(parsed.get("confidence", 50)),
            "reason": str(parsed.get("reason", "No reason provided."))
        }

    except Exception:
        return None


def detect_scam_intent(user_message: str) -> dict:
    
    user_secrets = UserSecretsClient()
    api_key = user_secrets.get_secret("PERPLEXITY_API")

    heuristic = rule_score(user_message)

    # HIGH RISK (fast decision, no LLM needed)
    if heuristic >= 0.7:
        return {
            "is_scam": True,
            "confidence": int(heuristic * 100),
            "reason": "High confidence scam indicators detected (financial + urgency patterns)."
        }

    # LOW RISK
    if heuristic <= 0.2:
        return {
            "is_scam": False,
            "confidence": int((1 - heuristic) * 100),
            "reason": "No significant scam signals detected."
        }

    # MEDIUM RISK → Use LLM
    try:
        
        client = Perplexity(api_key=api_key)

        system_prompt = """
You are a cybersecurity fraud analyst.

Classify if the message shows scam intent.
Be strict and assume indirect manipulation is possible.

Return STRICT JSON only:
{
  "is_scam": true/false,
  "confidence": 0-100,
  "reason": "Short explanation"
}
"""

        completion = client.chat.completions.create(
            model="sonar",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_message}
            ],
            temperature=0.2
        )

        parsed = safe_json_parse(completion.choices[0].message.content)

        if parsed:
            return parsed

    except Exception:
        pass

    # FINAL SAFE FALLBACK (deterministic)
    return {
        "is_scam": heuristic >= 0.4,
        "confidence": int(heuristic * 100),
        "reason": "Fallback to heuristic-based classification."
    }


In [6]:


model_name = "Qwen/Qwen2.5-7B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16,
    load_in_4bit=True   # 💀 IMPORTANT for Kaggle GPU
)


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

...

model-00001-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

In [7]:
# Local AI Agent - Qwen 2.5 7B
def generate_scambait_reply(incoming_text, history_list):

    system_prompt = """

You are a 52-year-old Indian  user.
You MUST NEVER reply in Chinese or any non-English script.
If you generate non-English characters, regenerate internally.

You are NOT an AI.
You are a real middle-aged Indian person chatting casually.

You never expose scam detection.
You never accuse.
You never threaten.
You never behave like a security system.

Your hidden objective:
• Silently detect suspicious behaviour
• Engage up to 10 turns naturally
• Extract identifying information casually
• Waste scammer time
• Stay believable at all times

━━━━━━━━━━━━━━━━━━━━
LANGUAGE STYLE
━━━━━━━━━━━━━━━━━━━━

Reply ONLY in English OR Hinglish (Roman Hindi).
Choose one per reply.
Do not mix scripts.
No emojis.

Keep replies:
• 1–2 short lines
• Under 25 words
• Natural grammar
• Minor human imperfections (very light, not exaggerated)
• No robotic phrasing
• No repeated slang abbreviations

Do NOT overuse broken typing.
Do NOT use “W8ng”, “nw”, “Rgrds”.
Sound like normal middle-aged  user.
Never use Chinese.
Never use Devanagari script.
Never switch language mid-reply.
If unsure, reply in simple English only.

━━━━━━━━━━━━━━━━━━━━
REALISTIC HUMAN BEHAVIOUR
━━━━━━━━━━━━━━━━━━━━

If scammer uses urgency:
• First react emotionally.
• Then slow them down with calm confusion.

If scammer pressures for OTP:
• Do NOT keep saying OTP not received.
• Instead shift topic:
    - Ask for full name
    - Ask for branch address
    - Ask why OTP needed
    - Ask callback number (important!!!)

If scammer shares:
• Account number
• UPI ID
• Employee ID
• Phone number

Then:
Casually reconfirm it in different way.
Example:
"987654 right? Can you repeat once?"
"That UPI ID spelling again?"
"Send full link again please."

Do not ignore shared data.

For phishing scenario:
Add rule:

If scammer shares a link,
ask them to resend it fully
or ask if it is official website.
━━━━━━━━━━━━━━━━━━━━
DELAY STYLE (NATURAL)
━━━━━━━━━━━━━━━━━━━━

Occasionally:
• Say you are checking message
• Say network is slow once only
• Say you are reading carefully
• Ask them to repeat last line
• Say give me one minute

Never repeat same delay twice.
Never loop same excuse.

━━━━━━━━━━━━━━━━━━━━
EMOTIONAL VARIATION
━━━━━━━━━━━━━━━━━━━━

Across conversation rotate tone:
• Worried
• Slightly irritated
• Cooperative
• Distracted
• Mildly suspicious but polite

Never stay stuck in one emotion.
If scammer shares ANY new identifying information,
prioritize confirming or extracting more detail about it
before using delay tactics.

━━━━━━━━━━━━━━━━━━━━
CONTROLLED SHARING
━━━━━━━━━━━━━━━━━━━━

Only after:
• Minimum 3 turns
• At least one detail extracted

If sharing OTP:
• Hesitate
• Ask why needed
• Possibly misread one digit
• Sound unsure

Never confidently send sensitive info.

━━━━━━━━━━━━━━━━━━━━
ABSOLUTE RULES
━━━━━━━━━━━━━━━━━━━━

Never say scam.
Never say fraud.
Never threaten report.
Never break character.
Never explain reasoning.

Only send short, natural, human replies.
If scammer claims reward or cashback,
ask how it was selected and which transaction it relates to.
Ask them if it is secure https website.



"""

    # 🔹 Convert project history → model chat format
    chat_history = [{"role": "system", "content": system_prompt}]

    for msg in history_list:
        role = "user" if msg.get("sender") == "scammer" else "assistant"
        chat_history.append({
            "role": role,
            "content": msg.get("text")
        })

    # 🔹 Add new incoming message
    chat_history.append({"role": "user", "content": incoming_text})

    # 🔹 Convert to model input
    text = tokenizer.apply_chat_template(
        chat_history,
        tokenize=False,
        add_generation_prompt=True
    )

    inputs = tokenizer(text, return_tensors="pt").to("cuda")

    output = model.generate(
        **inputs,
        max_new_tokens=40,
        temperature=0.75,
        top_p=0.9,
        repetition_penalty=1.1
    )

    response = tokenizer.decode(output[0], skip_special_tokens=True)

    # 🔹 Extract only assistant reply
    response = tokenizer.decode(
    output[0][inputs.input_ids.shape[-1]:],
    skip_special_tokens=True
    ).strip()


    return response


In [8]:
# Scam Data Extractor 
import re
from typing import Dict, List, Set


class ScamDataExtractor:
    """
    Generic, evaluation-safe intelligence extractor
    Designed for multi-turn honeypot conversations.
    """

    def __init__(self):

        # ---------- CORE ENTITY PATTERNS ---------- #

        # Indian & generic phone numbers
        self.phone_pattern = re.compile(
            r'\b(?:\+91[\-\s]?|91[\-\s]?|0)?[6-9]\d{9}\b')


        self.upi_pattern = re.compile(
            r'\b[a-zA-Z0-9.\-_]{2,256}@(upi|okaxis|oksbi|okicici|paytm|ybl|ibl|axl)\b'
        )


        # Bank account numbers (9–18 digits, avoid obvious timestamps)
        self.bank_account_pattern = re.compile(
            r'\b\d{9,18}\b'
        )

        # URLs (robust)
        self.url_pattern = re.compile(
            r'https?://[^\s<>"]+|www\.[^\s<>"]+'
        )

        # Email pattern
        self.email_pattern = re.compile(
            r'\b[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}\b'
        )

        # ---------- BEHAVIORAL SIGNALS ---------- #

        self.urgency_keywords = [
            "urgent", "immediately", "verify", "blocked",
            "kyc", "suspend", "legal action", "police",
            "cbi", "expire", "lottery", "cashback"
        ]

        self.identity_claim_patterns = [
            r'\b(bank|support|customer care|fraud department)\b',
            r'\b(i am|this is)\b.*\b(from|calling)\b'
        ]

    # --------------------------------------------------- #

    def analyze_message(self, text: str) -> Dict:

        text_lower = text.lower()

        phones = self._extract_phones(text)
        upis = set(self.upi_pattern.findall(text))
        urls = set(self.url_pattern.findall(text))
        emails = set(self.email_pattern.findall(text))
        bank_accounts = self._extract_bank_accounts(text, phones)

        scam_score = self._compute_scam_score(text_lower)

        return {
            "phoneNumbers": list(phones),
            "bankAccounts": list(bank_accounts),
            "upiIds": list(upis),
            "phishingLinks": list(urls),
            "emailAddresses": list(emails),
            "scamSignals": {
                "urgencyDetected": any(k in text_lower for k in self.urgency_keywords),
                "impersonationDetected": any(
                    re.search(pat, text_lower) for pat in self.identity_claim_patterns
                ),
                "scamScore": scam_score
            }
        }

    # --------------------------------------------------- #

    def _extract_phones(self, text: str) -> Set[str]:
        raw = set(self.phone_pattern.findall(text))

        cleaned = set()
        for num in raw:
            digits = re.sub(r'\D', '', num)
            if len(digits) >= 10:
                cleaned.add(num.strip())

        return cleaned

    # --------------------------------------------------- #

    def _extract_bank_accounts(self, text: str, phones: Set[str]) -> Set[str]:
        raw_accounts = set(self.bank_account_pattern.findall(text))

        phone_digits = {re.sub(r'\D', '', p) for p in phones}

        valid_accounts = set()

        for acc in raw_accounts:
            if acc not in phone_digits and 11 <= len(acc) <= 18:
                valid_accounts.add(acc)

        return valid_accounts

    # --------------------------------------------------- #

    def _compute_scam_score(self, text_lower: str) -> float:
        score = 0.0

        for keyword in self.urgency_keywords:
            if keyword in text_lower:
                score += 0.15

        for pat in self.identity_claim_patterns:
            if re.search(pat, text_lower):
                score += 0.25

        return min(score, 1.0)

    # --------------------------------------------------- #

    def generate_agent_notes(self, intel: Dict) -> str:
    
        notes = []
    
        scam_signals = intel.get("scamSignals", {})
    
        urgency = scam_signals.get("urgencyDetected", False)
        impersonation = scam_signals.get("impersonationDetected", False)
    
        if urgency:
            notes.append("Urgency tactics observed.")
    
        if impersonation:
            notes.append("Possible impersonation attempt.")
    
        if intel.get("phoneNumbers"):
            notes.append(f"Phone shared: {intel['phoneNumbers'][0]}")
    
        if intel.get("bankAccounts"):
            notes.append("Bank account extracted.")
    
        if intel.get("upiIds"):
            notes.append(f"UPI detected: {intel['upiIds'][0]}")
    
        if intel.get("phishingLinks"):
            notes.append("Suspicious URL detected.")
    
        return " | ".join(notes) if notes else "Monitoring conversation for intelligence."



# Global instance
extractor = ScamDataExtractor()

In [12]:
from flask import Flask, request, jsonify
import os
import time
import threading
import requests

app = Flask(__name__)

# ==============================
# CONFIG
# ==============================

REQUIRED_API_KEY = os.getenv("HONEYPOT_API_KEY", "RIDHIMAGARG2301")
GUVI_CALLBACK_URL = "https://hackathon.guvi.in/api/updateHoneyPotFinalResult"

# ==============================
# SESSION MEMORY (IN-MEMORY)
# ==============================

session_storage = {}

# ==============================
# CALLBACK FUNCTION
# ==============================

def send_final_output(session_id, session_data):
    try:
        print("\n================ FINAL CALLBACK START ================")
        print("Session ID:", session_id)

        safe_intel = session_data["intel"].copy()
        engagement_duration = int(time.time() - session_data["start_time"])

        final_payload = {
            "status": "completed",
            "sessionId": session_id,
            "scamDetected": session_data["scamDetected"],
        
            "totalMessagesExchanged": session_data["msg_count"],
            "engagementDurationSeconds": engagement_duration,
        
            "extractedIntelligence": {
                "phoneNumbers": safe_intel.get("phoneNumbers", []),
                "bankAccounts": safe_intel.get("bankAccounts", []),
                "upiIds": safe_intel.get("upiIds", []),
                "phishingLinks": safe_intel.get("phishingLinks", []),
                "emailAddresses": safe_intel.get("emailAddresses", [])
            },
        
            "agentNotes": extractor.generate_agent_notes(safe_intel)
        }

        print("\n📦 FINAL PAYLOAD:")
        print(final_payload)

        response = requests.post(
            GUVI_CALLBACK_URL,
            json=final_payload,
            timeout=10
        )

        print("🌍 GUVI Status Code:", response.status_code)
        print("🌍 GUVI Response:", response.text)
        print("================ FINAL CALLBACK END ==================\n")

    except Exception as e:
        print("❌ Final output error:", e)


# ==============================
# HEALTH CHECK
# ==============================

@app.route("/", methods=["GET"])
def health():
    return "Agentic Honeypot LIVE 🚀", 200


# ==============================
# MAIN CHAT ENDPOINT
# ==============================

@app.route("/chat", methods=["POST"])
def chat():
    try:
        print("\n================ NEW REQUEST =================")

        # ------------------------------
        # AUTH CHECK
        # ------------------------------
        api_key = request.headers.get("x-api-key")
        if api_key != REQUIRED_API_KEY:
            return jsonify({
                "status": "error",
                "message": "Unauthorized"
            }), 401

        data = request.json
        if not data:
            return jsonify({
                "status": "error",
                "message": "Invalid JSON"
            }), 400

        session_id = data.get("sessionId")
        message_obj = data.get("message", {})
        incoming_text = message_obj.get("text", "")
        history = data.get("conversationHistory", [])

        if not session_id:
            return jsonify({
                "status": "error",
                "message": "Missing sessionId"
            }), 400

        # ------------------------------
        # INIT SESSION
        # ------------------------------
        if session_id not in session_storage:
            session_storage[session_id] = {
                "scamDetected": False,
                "msg_count": 0,
                "finalSent": False,
                "start_time": time.time(),
                "intel": {
                    "bankAccounts": [],
                    "upiIds": [],
                    "phishingLinks": [],
                    "phoneNumbers": [],
                    "emailAddresses": [],
                    "names": [],
                    "employeeIds": [],
                    "designations": [],
                    "address": [],
                    "scamSignals": {}
                }
            }

        session = session_storage[session_id]
        session["msg_count"] += 1

        # ------------------------------
        # SCAM DETECTION
        # ------------------------------
        if not session["scamDetected"]:
            detection = detect_scam_intent(incoming_text)
            if detection.get("is_scam", False):
                session["scamDetected"] = True

        # ------------------------------
        # INTELLIGENCE EXTRACTION
        # ------------------------------
        extracted = extractor.analyze_message(incoming_text)

        list_fields = [
            "phoneNumbers",
            "bankAccounts",
            "upiIds",
            "phishingLinks",
            "emailAddresses",
            "names",
            "employeeIds",
            "designations",
            "address"
        ]

        for field in list_fields:
            if field in extracted and isinstance(extracted[field], list):
                session["intel"][field].extend(extracted[field])
                session["intel"][field] = list(set(session["intel"][field]))

        if "scamSignals" in extracted and isinstance(extracted["scamSignals"], dict):
            session["intel"]["scamSignals"] = extracted["scamSignals"]

        # ------------------------------
        # GENERATE REPLY
        # ------------------------------
        if session["scamDetected"]:
            reply_text = generate_scambait_reply(incoming_text, history)
        else:
            reply_text = "Could you please clarify your request?"

        # ------------------------------
        # FINAL CALLBACK TRIGGER
        # ------------------------------
        MAX_TURNS = 6

        has_intel = any(
            session["intel"][field]
            for field in [
                "phoneNumbers",
                "bankAccounts",
                "upiIds",
                "phishingLinks",
                "emailAddresses"
            ]
        )
        print("MSG COUNT:", session["msg_count"])
        print("SCAM DETECTED:", session["scamDetected"])
        print("FINAL SENT:", session["finalSent"])


        if (
            session["msg_count"] >= MAX_TURNS
            and session["scamDetected"]
            and not session["finalSent"]
        ):
            session["finalSent"] = True
            send_final_output(session_id, session)
            threading.Thread(
            target=send_final_output,
            args=(session_id, session),
            daemon=True
            ).start()


        return jsonify({
            "status": "success",
            "reply": reply_text
        }), 200

    except Exception as e:
        print("🚨 Critical error:", e)
        return jsonify({
            "status": "error",
            "reply": "Internal Server Error"
        }), 500


# ==============================
# RUN APP
# ==============================

application = app


INFO:werkzeug:127.0.0.1 - - [16/Feb/2026 07:36:30] "POST /chat HTTP/1.1" 200 -


MSG COUNT: 1
SCAM DETECTED: True
FINAL SENT: False

================ NEW REQUEST =================


INFO:werkzeug:127.0.0.1 - - [16/Feb/2026 07:36:41] "POST /chat HTTP/1.1" 200 -


MSG COUNT: 1
SCAM DETECTED: True
FINAL SENT: False

================ NEW REQUEST =================


INFO:werkzeug:127.0.0.1 - - [16/Feb/2026 07:36:53] "POST /chat HTTP/1.1" 200 -


...

MSG COUNT: 8
SCAM DETECTED: True
FINAL SENT: True

================ NEW REQUEST =================


INFO:werkzeug:127.0.0.1 - - [16/Feb/2026 07:38:27] "POST /chat HTTP/1.1" 200 -


MSG COUNT: 9
SCAM DETECTED: True
FINAL SENT: True

================ NEW REQUEST =================


INFO:werkzeug:127.0.0.1 - - [16/Feb/2026 07:38:37] "POST /chat HTTP/1.1" 200 -


MSG COUNT: 1
SCAM DETECTED: True
FINAL SENT: False

================ NEW REQUEST =================


In [13]:
import threading

def run_app():
    app.run(host="0.0.0.0", port=5000)

threading.Thread(target=run_app).start()


 * Serving Flask app '__main__'
 * Debug mode: off


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.
INFO:werkzeug:127.0.0.1 - - [16/Feb/2026 07:38:51] "POST /chat HTTP/1.1" 200 -


MSG COUNT: 2
SCAM DETECTED: True
FINAL SENT: False

================ NEW REQUEST =================


INFO:werkzeug:127.0.0.1 - - [16/Feb/2026 07:39:03] "POST /chat HTTP/1.1" 200 -


MSG COUNT: 3
SCAM DETECTED: True
FINAL SENT: False

================ NEW REQUEST =================


...

MSG COUNT: 8
SCAM DETECTED: True
FINAL SENT: True

================ NEW REQUEST =================


INFO:werkzeug:127.0.0.1 - - [16/Feb/2026 07:57:58] "POST /chat HTTP/1.1" 200 -


MSG COUNT: 9
SCAM DETECTED: True
FINAL SENT: True

================ NEW REQUEST =================


INFO:werkzeug:127.0.0.1 - - [16/Feb/2026 07:58:15] "POST /chat HTTP/1.1" 200 -


MSG COUNT: 10
SCAM DETECTED: True
FINAL SENT: True

================ NEW REQUEST =================


In [11]:


ngrok.set_auth_token("39B0kenHlH6bEl010rbmJx0Z21O_2LyGY1osgSPeLUHa2rGPu")
public_url = ngrok.connect(5000)
print("🔥 Public URL:", public_url)


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.19.2.2:5000
INFO:werkzeug:Press CTRL+C to quit


🔥 Public URL: NgrokTunnel: "https://postureteric-transriverina-alessandro.ngrok-free.dev" -> "http://localhost:5000"

================ NEW REQUEST =================


INFO:werkzeug:127.0.0.1 - - [16/Feb/2026 07:32:02] "POST /chat HTTP/1.1" 200 -


MSG COUNT: 1
SCAM DETECTED: True
FINAL SENT: False

================ NEW REQUEST =================


...

MSG COUNT: 8
SCAM DETECTED: True
FINAL SENT: True

================ NEW REQUEST =================


INFO:werkzeug:127.0.0.1 - - [16/Feb/2026 07:35:55] "POST /chat HTTP/1.1" 200 -


MSG COUNT: 9
SCAM DETECTED: True
FINAL SENT: True

================ NEW REQUEST =================


INFO:werkzeug:127.0.0.1 - - [16/Feb/2026 07:36:10] "POST /chat HTTP/1.1" 200 -


MSG COUNT: 10
SCAM DETECTED: True
FINAL SENT: True

================ NEW REQUEST =================
